In [1]:
import torch
import torch.nn as nn
import csv
from torch import optim as optim
import torch.utils.data as data
import math
import copy

In [81]:
with open('C:\Mithra\Extra Curricular\WNCC Assignment\queries.csv', 'r') as file:
    csvfile = csv.reader(file)
    dataset = {}
    for line in csvfile:
        dataset[line[0]] = line[1]
tr = list(dataset.keys())
tr.remove("QUERY")
outputs = list(dataset.values())
outputs.remove("CATEGORY")
ev = {}
print(tr)

max_seq_length = max(len(clean(s).split()) for s in tr)

['What is overfitting and underfitting?', 'What is the best site to practice CP?', 'What laptop is good for coding?', 'What is the difference between HTML and HTML5?', 'How do I solve the knapsack problem?', 'How do I detect a cycle in a graph?', 'How to analyze complexity in CP?', 'What are some useful STL functions?', 'How do I write a good README?', 'How do I balance college and tech learning?', 'What is open source licensing?', 'How do I connect frontend and backend?', 'How do I host a website on GitHub Pages?', 'How do I stay consistent with coding practice?', 'How do I prepare for tech interviews?', 'What are common ML algorithms?', 'How can I build a good portfolio?', 'How do I find tech internships?', 'What is dynamic programming?', 'What is continuous integration in open source?', 'How do I implement a binary search tree?', 'How do I tune hyperparameters?', 'How do I implement linear regression?', 'What is the role of a loss function?', 'What is Git and why is it used?', "What

In [82]:
for i, key in enumerate(set(outputs)):
  ev[key] = i
#ev.pop('CATEGORY')
print(ev)

{'Open Source': 0, 'DSA': 1, 'Web Development': 2, 'Machine Learning': 3, 'Competitive Programming': 4, 'General': 5}


In [213]:
def clean(sentence):
    words = sentence.split()
    length = len(words)
    l = len(words[length-1])
    for i in range(len(words)):
        words[i] = words[i].lower()
        if '-' in words[i]:
            tmp = []
            without = words[i].split('-')
            tmp = words[:i]+without+words[i+1:]
            words = tmp
    if words[-1][-1] == "?":
        words[-1] = words[-1][0:l-1]
    for word in words:
        if word in ( "a", "an", "the", "is", "are", "was", "were", "in", "on", "at", "of", "for", "to", "from", "that", "which", "who", "what", "when", "where", "how", "do", "does", "did", "with","and", "or", "but", "if", "then", "so", "not", "be", "been", "being", "have", "has", "had","can", "could", "will", "would", "should", "may", "might", "must", "i"):
            words.remove(word)
    return " ".join(words)

In [214]:
print(clean("What are some beginner-friendly open-source projects to contribute to?"))

are some beginner friendly open source projects contribute


In [215]:
word_to_id = {"<PAD>":0, "<UNK>":1}
id_to_word = {0:"<PAD>", 1:"UNK"}
idx = 2

for sentence in tr:
    for word in clean(sentence).lower().split():
        if word not in word_to_id:
            word_to_id[word] = idx
            id_to_word[idx] = word
            idx += 1

def tokenize(sentence,max_seq_length):
    tokens=[]
    for word in clean(sentence).split():
        tokens.append(word_to_id.get(word,1))
    if len(tokens) != max_seq_length:
        no_of_padding = max_seq_length - len(tokens)
        padding = [0]*no_of_padding                              #I had a problem with dimensions so i added passing to make all sentences of equal length
        tokens.extend(padding)
    token_ids = torch.tensor([tokens])
    return token_ids

l = []
for sentence in tr:
    l.append(tokenize(sentence,max_seq_length))
src_data = torch.cat(l, dim=0)

l = []
for word in outputs:
    l.append(ev[word])
tgt_data = torch.tensor(l)

"""def embed(vocab_size, emb_dim, sentence):
    tokens=[]
    for word in clean(sentence).split():
        tokens.append(word_to_id.get(word,1))
    embedding_layer = nn.Embedding(vocab_size,emb_dim)              #torch.nn.Embedding(num_embeddings, embedding_dim, padding_idx=None, max_norm=None, norm_type=2.0, scale_grad_by_freq=False, sparse=False, _weight=None, _freeze=False, device=None, dtype=None)[source]
    token_ids = torch.tensor([tokens])
    print(token_ids.shape)
    embedded_output = embedding_layer(token_ids)
    print(embedded_output.shape)
    return embedded_output"""

'def embed(vocab_size, emb_dim, sentence):\n    tokens=[]\n    for word in clean(sentence).split():\n        tokens.append(word_to_id.get(word,1))\n    embedding_layer = nn.Embedding(vocab_size,emb_dim)              #torch.nn.Embedding(num_embeddings, embedding_dim, padding_idx=None, max_norm=None, norm_type=2.0, scale_grad_by_freq=False, sparse=False, _weight=None, _freeze=False, device=None, dtype=None)[source]\n    token_ids = torch.tensor([tokens])\n    print(token_ids.shape)\n    embedded_output = embedding_layer(token_ids)\n    print(embedded_output.shape)\n    return embedded_output'

In [216]:
"""l = []
for word in outputs:
    l.append(tokenize(word))
tgt_data = torch.cat(l,dim=0)
print(l)"""

'l = []\nfor word in outputs:\n    l.append(tokenize(word))\ntgt_data = torch.cat(l,dim=0)\nprint(l)'

In [141]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [179]:
A= torch.randn(5,10,8)          
A=A.view(5,2,10,4)   #basically 5 sentences, 2 heads, 10 words each, dimension of each head = 4
print(A.transpose(-2,-1).shape)
B = torch.matmul(A,A.transpose(-2,-1))
print(B.shape)

torch.Size([5, 2, 4, 10])
torch.Size([5, 2, 10, 10])


In [180]:
C=torch.matmul(B, A)
print(C.shape)

torch.Size([5, 2, 10, 4])


In [171]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [172]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [165]:
position = torch.arange(0, 5, dtype=torch.float).unsqueeze(1)
position

tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.]])

In [144]:
"""max_seq_length = 10
d_model = 64
pe = torch.zeros(max_seq_length, d_model)
position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
print(position)
print(div_term)
print(position*div_term)
print(torch.sin(position * div_term).shape)
print(torch.cos(position * div_term))
print(pe[:, 0::2].shape)
print(pe[:, 1::2].shape)"""

'max_seq_length = 10\nd_model = 64\npe = torch.zeros(max_seq_length, d_model)\nposition = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)\ndiv_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))\nprint(position)\nprint(div_term)\nprint(position*div_term)\nprint(torch.sin(position * div_term).shape)\nprint(torch.cos(position * div_term))\nprint(pe[:, 0::2].shape)\nprint(pe[:, 1::2].shape)'

In [145]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [146]:
"""class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x"""

'class DecoderLayer(nn.Module):\n    def __init__(self, d_model, num_heads, d_ff, dropout):\n        super(DecoderLayer, self).__init__()\n        self.self_attn = MultiHeadAttention(d_model, num_heads)\n        self.cross_attn = MultiHeadAttention(d_model, num_heads)\n        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)\n        self.norm1 = nn.LayerNorm(d_model)\n        self.norm2 = nn.LayerNorm(d_model)\n        self.norm3 = nn.LayerNorm(d_model)\n        self.dropout = nn.Dropout(dropout)\n\n    def forward(self, x, enc_output, src_mask, tgt_mask):\n        attn_output = self.self_attn(x, x, x, tgt_mask)\n        x = self.norm1(x + self.dropout(attn_output))\n        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)\n        x = self.norm2(x + self.dropout(attn_output))\n        ff_output = self.feed_forward(x)\n        x = self.norm3(x + self.dropout(ff_output))\n        return x'

In [204]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        #self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        #self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src):  #, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        #tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        #seq_length = tgt.size(1)
        #nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        #tgt_mask = tgt_mask & nopeak_mask
        return src_mask #, tgt_mask

    def forward(self, src): #, tgt):
        src_mask = self.generate_mask(src)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        #tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        """dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)"""

        pooled = enc_output.mean(dim=1)
        output = self.fc(pooled)
        return output

In [205]:
V = torch.randn(5,10,8)
W = V.mean(dim=2)
print(W.shape)

torch.Size([5, 10])


In [206]:
"""x = torch.randn(2,4,8)
print(x)
fc = nn.Linear(8, 6)
x = x.mean(dim=1)
print(x)
print(x.shape)"""

'x = torch.randn(2,4,8)\nprint(x)\nfc = nn.Linear(8, 6)\nx = x.mean(dim=1)\nprint(x)\nprint(x.shape)'

In [301]:
src_vocab_size = len(word_to_id)
tgt_vocab_size = 6
d_model = 512
num_heads = 16
num_layers = 6
d_ff = 2048
dropout = 0.2

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

In [302]:
print(tgt_data.shape)

torch.Size([328])


In [303]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()
for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data)
    #print(output.shape)
    #print(output)
    #print(output.contiguous().view(-1, tgt_vocab_size))
    loss = criterion(output, tgt_data)
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 1.7623809576034546
Epoch: 2, Loss: 1.765493631362915
Epoch: 3, Loss: 1.5453052520751953
Epoch: 4, Loss: 1.5205286741256714
Epoch: 5, Loss: 1.2736605405807495
Epoch: 6, Loss: 1.195817232131958
Epoch: 7, Loss: 1.1690993309020996
Epoch: 8, Loss: 1.1120045185089111
Epoch: 9, Loss: 0.9791858196258545
Epoch: 10, Loss: 0.8450057506561279
Epoch: 11, Loss: 0.8135510087013245
Epoch: 12, Loss: 0.7711024284362793
Epoch: 13, Loss: 0.6499886512756348
Epoch: 14, Loss: 0.560408890247345
Epoch: 15, Loss: 0.5222970247268677
Epoch: 16, Loss: 0.4860163629055023
Epoch: 17, Loss: 0.42542603611946106
Epoch: 18, Loss: 0.3811836540699005
Epoch: 19, Loss: 0.2979271709918976
Epoch: 20, Loss: 0.2951813042163849
Epoch: 21, Loss: 0.2822270691394806
Epoch: 22, Loss: 0.2110579013824463
Epoch: 23, Loss: 0.17536652088165283
Epoch: 24, Loss: 0.1704508364200592
Epoch: 25, Loss: 0.14611166715621948
Epoch: 26, Loss: 0.11964844912290573
Epoch: 27, Loss: 0.0966193750500679
Epoch: 28, Loss: 0.08029880374670029

In [305]:
q = ["How do I implement Dijkstra’s algorithm in Python?", "What is the difference between CNN and RNN?", "How do I deploy a React app on Vercel?", "What are some beginner-friendly open-source projects to contribute to?", "Where can I find past WnCC event recordings?"]
l = []
for sentence in q:
    l.append(tokenize(sentence,max_seq_length))
val_src_data = torch.cat(l, dim=0)

out = ["DSA", "Machine Learning", "Web Development", "Open Source", "General"]
l = []
for word in out:
    l.append(ev[word])
val_tgt_data = torch.tensor(l)

transformer.eval()
with torch.no_grad():
    output = transformer(val_src_data)  # shape: [no_of_sentences, no_of_classes)
    predicted_class = torch.argmax(output, dim=1)  #shape: [no_of_sentences]

print(predicted_class)
tmp = []
for pred in predicted_class:
    tmp.append(idx_to_label[pred.item()])
predicted_class=tmp
print(predicted_class)

idx_to_label = {} #{'Open Source': 0, 'DSA': 1, 'Web Development': 2, 'Machine Learning': 3, 'Competitive Programming': 4, 'General': 5}
for key, value in ev.items():
    idx_to_label[value] = key
for i, pred in enumerate(predicted_class):
    print(f"{q[i]} → {pred}")

tensor([1, 3, 2, 4, 5])
['DSA', 'Machine Learning', 'Web Development', 'Competitive Programming', 'General']
How do I implement Dijkstra’s algorithm in Python? → DSA
What is the difference between CNN and RNN? → Machine Learning
How do I deploy a React app on Vercel? → Web Development
What are some beginner-friendly open-source projects to contribute to? → Competitive Programming
Where can I find past WnCC event recordings? → General


In [306]:
print(out, predicted_class)

['DSA', 'Machine Learning', 'Web Development', 'Open Source', 'General'] ['DSA', 'Machine Learning', 'Web Development', 'Competitive Programming', 'General']


In [307]:
print (idx_to_label)

{0: 'Open Source', 1: 'DSA', 2: 'Web Development', 3: 'Machine Learning', 4: 'Competitive Programming', 5: 'General'}


In [308]:
"""transformer.eval()

ev = ["How do I implement Dijkstra’s algorithm in Python?", "What is the difference between CNN and RNN?", "How do I deploy a React app on Vercel?", "What are some beginner-friendly open-source projects to contribute to?", "Where can I find past WnCC event recordings?"]
l = []
for sentence in ev:
    l.append(tokenize(sentence))
val_src_data = torch.cat(l, dim=0)

l = []
for word in outputs:
    l.append(tokenize(word))
tgt_data = torch.cat(l,dim=0)


with torch.no_grad():

    val_output = transformer.forward()
    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), val_tgt_data[:, 1:].contiguous().view(-1))
    print(f"Validation Loss: {val_loss.item()}")"""

'transformer.eval()\n\nev = ["How do I implement Dijkstra’s algorithm in Python?", "What is the difference between CNN and RNN?", "How do I deploy a React app on Vercel?", "What are some beginner-friendly open-source projects to contribute to?", "Where can I find past WnCC event recordings?"]\nl = []\nfor sentence in ev:\n    l.append(tokenize(sentence))\nval_src_data = torch.cat(l, dim=0)\n\nl = []\nfor word in outputs:\n    l.append(tokenize(word))\ntgt_data = torch.cat(l,dim=0)\n\n\nwith torch.no_grad():\n\n    val_output = transformer.forward()\n    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), val_tgt_data[:, 1:].contiguous().view(-1))\n    print(f"Validation Loss: {val_loss.item()}")'

In [309]:
print(idx_to_label)

{0: 'Open Source', 1: 'DSA', 2: 'Web Development', 3: 'Machine Learning', 4: 'Competitive Programming', 5: 'General'}


In [310]:
from collections import Counter
print(Counter(outputs))

Counter({'Machine Learning': 66, 'Web Development': 63, 'Competitive Programming': 55, 'DSA': 54, 'Open Source': 50, 'General': 40})


In [311]:
print(ev)

{'Open Source': 0, 'DSA': 1, 'Web Development': 2, 'Machine Learning': 3, 'Competitive Programming': 4, 'General': 5}


In [312]:
no_of_correct_predictions = 0
for i in range(len(out)):
    if out[i] == predicted_class[i]:
        no_of_correct_predictions+=1
Accuracy = no_of_correct_predictions / len(out)
print("Accuracy: ", Accuracy)

Accuracy:  0.8


In [313]:
precisions = []
recalls = []
F1_scores = []

In [314]:
def compare(label, pred, tgt):
    TP=FP=FN=0
    for i in range(len(tgt)):
        if pred[i]==label:
            if pred[i]==tgt[i]:
                TP+=1
            else:
                FP+=1
        elif tgt[i]==label:
            FN+=1
    if TP+FP+FN==0:
        print(f"Can't evaluate due to non availability of data for {label}")
        Precision = Recall = F1
    if TP+FP==0:
        print(f"Precision and F1 of {label} can't be calculated")
        Recall = TP/(TP+FN)
        print(f"Recall of {label}: {Recall}")
        Precision = F1 = 0
    elif TP+FN ==0 :
        print(f"Recall and F1 of {label} can't be calculated")
        Precision = TP/(TP+FP)
        print(f"Precision of {label}: {Precision}") 
        Recall = F1 = 0
    else:
        Precision = TP/(TP+FP)
        Recall = TP/(TP+FN)
        F1 = 2*(Precision*Recall)/(Precision+Recall)
        print(f"Precision of {label}: {Precision}") 
        print(f"Recall of {label}: {Recall}") 
        print(f"F1 of {label}: {F1}") 
    return Precision, Recall, F1

In [315]:
for key in ev:
    precision, recall, F1 = compare(key, predicted_class, out)
    precisions.append(precision)
    recalls.append(recall)
    F1_scores.append(F1)

Precision and F1 of Open Source can't be calculated
Recall of Open Source: 0.0
Precision of DSA: 1.0
Recall of DSA: 1.0
F1 of DSA: 1.0
Precision of Web Development: 1.0
Recall of Web Development: 1.0
F1 of Web Development: 1.0
Precision of Machine Learning: 1.0
Recall of Machine Learning: 1.0
F1 of Machine Learning: 1.0
Recall and F1 of Competitive Programming can't be calculated
Precision of Competitive Programming: 0.0
Precision of General: 1.0
Recall of General: 1.0
F1 of General: 1.0


In [316]:
print(recalls)

[0.0, 1.0, 1.0, 1.0, 0, 1.0]


In [317]:
Macro_Precision = sum(precisions)/len(precisions)
Macro_Recall = sum(recalls)/len(recalls)
MacroF1 = sum(F1_scores)/len(F1_scores)

In [318]:
print("Accuracy: ", Accuracy)
print("Precision: ", Macro_Precision)
print("Recall: ", Macro_Recall)
print("F1 Score: ", MacroF1)

Accuracy:  0.8
Precision:  0.6666666666666666
Recall:  0.6666666666666666
F1 Score:  0.6666666666666666
